In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def solve_optimal_execution(T=24, S_bar=1.0, rho=0.99, sigma=0.01, lambda_=0.01, X_total=1.0):
    N_s = 100  # Price grid points
    N_x = 50   # Shares grid points
    N_trades = 50  # Number of possible trade sizes
    
    price_std = sigma / np.sqrt(1 - rho**2)
    S_grid = np.linspace(S_bar - 4*price_std, S_bar + 4*price_std, N_s)
    X_grid = np.linspace(0, X_total, N_x)
    
    V = np.zeros((T+1, N_s, N_x))
    policy = np.zeros((T, N_s, N_x))
    
    # Transition matrix
    P = np.zeros((N_s, N_s))
    for i in range(N_s):
        mean = rho * S_grid[i] + (1-rho) * S_bar
        P[i,:] = np.exp(-0.5 * ((S_grid - mean)/sigma)**2)
        P[i,:] /= np.sum(P[i,:])
    
    # Plot transition matrix
    plt.figure(figsize=(7, 5))
    plt.imshow(P, cmap='hot', interpolation='nearest', 
               extent=[S_grid[0], S_grid[-1], S_grid[0], S_grid[-1]], 
               aspect='auto')
    plt.colorbar()
    plt.title('Price Transition Probability Matrix')
    plt.xlabel('Next Price')
    plt.ylabel('Current Price')
    plt.show()
    
    # Backward induction
    for t in range(T-1, -1, -1):
        for i_s, s in enumerate(S_grid):
            for i_x, x in enumerate(X_grid):
                if x == 0:
                    continue
                
                trades = np.linspace(0, x, N_trades)
                values = np.zeros_like(trades)
                
                for k, trade in enumerate(trades):
                    reward = trade * (s - lambda_ * trade)
                    
                    if t < T-1:
                        x_next_idx = np.searchsorted(X_grid, x - trade)
                        if x_next_idx < len(X_grid):
                            future_value = P[i_s] @ V[t+1, :, x_next_idx]
                            values[k] = reward + future_value
                    else:
                        values[k] = reward
                
                best_k = np.argmax(values)
                V[t, i_s, i_x] = values[best_k]
                policy[t, i_s, i_x] = trades[best_k]
    
    return V, policy, S_grid, X_grid, P

def get_analytical_solution(S_t: float, W_t: float, rho: float, lambda_: float, S_bar: float) -> float:
    """Analytical solution at T-2"""
    return (1/9) * W_t + (4*rho*(1 + rho - 0.5*rho**2)*(S_t - S_bar))/(9*lambda_)

def simulate_and_compare(V, policy, S_grid, X_grid, T, S_bar, rho, sigma, lambda_, n_sims=10000):
    S0 = 1.0
    uniform_trade = 1.0/T
    
    optimal_profits = np.zeros(n_sims)
    uniform_profits = np.zeros(n_sims)
    
    for i in range(n_sims):
        S = np.zeros(T+1)
        S[0] = S0
        
        for t in range(T):
            S[t+1] = rho * S[t] + (1-rho) * S_bar + sigma * np.random.randn()
        
        opt_profit = 0
        unif_profit = 0
        x_remaining = 1.0
        
        for t in range(T):
            s_idx = np.searchsorted(S_grid, S[t])
            x_idx = np.searchsorted(X_grid, x_remaining)
            
            trade = policy[t, min(s_idx, len(S_grid)-1), min(x_idx, len(X_grid)-1)]
            opt_profit += trade * (S[t] - lambda_ * trade)
            x_remaining -= trade
            
            unif_profit += uniform_trade * (S[t] - lambda_ * uniform_trade)
        
        optimal_profits[i] = opt_profit
        uniform_profits[i] = unif_profit
    
    return optimal_profits, uniform_profits

def main():
    T, S_bar, rho = 24, 1.0, 0.99
    sigma, lambda_ = 0.01, 0.01
    
    V, policy, S_grid, X_grid, P = solve_optimal_execution(T, S_bar, rho, sigma, lambda_)
    
    # Part (a): Policy at T-2
    print("\n(a) Policy at T-2:")
    t_minus_2_policy = policy[T-2, :, -1]  # Full shares remaining
    print(f"Max trade size: {np.max(t_minus_2_policy):.4f}")
    print(f"Min trade size: {np.min(t_minus_2_policy):.4f}")
    print(f"Mean trade size: {np.mean(t_minus_2_policy):.4f}")
    
    plt.figure(figsize=(8, 6))
    share_levels = [0.25, 0.5, 0.75, 1.0]
    for w in share_levels:
        w_idx = np.searchsorted(X_grid, w)
        plt.plot(S_grid, policy[T-2,:,w_idx], '--', label=f'W={w:.2f}')
    plt.title('T-2 Policy')
    plt.xlabel('Stock Price')
    plt.ylabel('Optimal Trade Size')
    plt.legend()
    plt.grid(True)
    plt.show()
    
    # Part (b): T/2 policy with 50% shares
    mid_t = T//2
    x_idx = len(X_grid)//2
    mid_policy = policy[mid_t, :, x_idx]
    
    print("\n(b) Policy at t=T/2 with 50% shares:")
    print(f"Max trade size: {np.max(mid_policy):.4f}")
    print(f"Min trade size: {np.min(mid_policy):.4f}")
    print(f"Mean trade size: {np.mean(mid_policy):.4f}")
    
    plt.figure(figsize=(8, 6))
    plt.plot(S_grid, mid_policy)
    plt.title(f'Optimal Trading at t={mid_t} with 50% Shares Left')
    plt.xlabel('Stock Price')
    plt.ylabel('Shares to Trade')
    plt.grid(True)
    plt.show()
    
    # Part (c): Policy vs shares at S=1
    s_idx = np.searchsorted(S_grid, 1.0)
    shares_policy = policy[mid_t, s_idx, :]
    
    print("\n(c) Policy at t=T/2 and S=1:")
    print(f"Max trade size: {np.max(shares_policy):.4f}")
    print(f"Min trade size: {np.min(shares_policy):.4f}")
    print(f"Trade size at 50% shares: {shares_policy[len(X_grid)//2]:.4f}")
    
    plt.figure(figsize=(8, 6))
    plt.plot(X_grid, shares_policy)
    plt.title(f'Optimal Trading at t={mid_t} and S=1')
    plt.xlabel('Shares Remaining')
    plt.ylabel('Shares to Trade')
    plt.grid(True)
    plt.show()
    
    # Part (d): Simulation comparison
    optimal_profits, uniform_profits = simulate_and_compare(
        V, policy, S_grid, X_grid, T, S_bar, rho, sigma, lambda_)
    
    print("\n(d) Simulation Results:")
    print(f"Average Optimal Profit: {np.mean(optimal_profits):.6f}")
    print(f"Average Uniform Profit: {np.mean(uniform_profits):.6f}")
    print(f"Improvement: {((np.mean(optimal_profits)/np.mean(uniform_profits) - 1)*100):.2f}%")
    print(f"Standard deviation - Optimal: {np.std(optimal_profits):.6f}")
    print(f"Standard deviation - Uniform: {np.std(uniform_profits):.6f}")
    print(f"\nRho Impact:")
    print(f"High rho ({rho}) allows optimal strategy to exploit price predictability")
    print(f"by timing trades according to mean-reverting price dynamics.")
    
    plt.figure(figsize=(8, 6))
    plt.hist(optimal_profits, bins=50, alpha=0.5, label='Optimal')
    plt.hist(uniform_profits, bins=50, alpha=0.5, label='Uniform')
    plt.xlabel('Profit')
    plt.ylabel('Frequency')
    plt.legend()
    plt.title('Profit Distribution Comparison')
    plt.show()

if __name__ == "__main__":
    main()

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

def simulate_sv_paths(S0, x0, x_bar, theta, gamma_x, rho, rF, muS, T, delta, Nsims):
    Nperiods = int(T/delta)
    S = np.full((Nsims, Nperiods + 1), S0)
    x = np.full((Nsims, Nperiods + 1), x0)
    
    S[:, 0] = S0
    x[:, 0] = x0
    
    for t in range(1, Nperiods + 1):
        eps_S, eps_v = np.random.normal(0, 1, (2, Nsims))
        vt = np.exp(x[:, t-1])
        eta_St = (muS - rF) / vt
        eta_x = -0.2
        
        x[:, t] = x[:, t-1] - theta * (x[:, t-1] - x_bar) * delta \
                 - (rho * eta_St + np.sqrt(1 - rho**2) * eta_x) * gamma_x * delta \
                 + gamma_x * np.sqrt(delta) * (rho * eps_S + np.sqrt(1 - rho**2) * eps_v)
        
        S[:, t] = S[:, t-1] * np.exp((rF - 0.5 * vt**2) * delta + vt * np.sqrt(delta) * eps_S)
    
    return S, x

def compute_realized_variance(S, window=10, delta=1/252):
    log_returns = np.diff(np.log(S), axis=1)
    RV = np.zeros(S.shape)
    
    for t in range(window, S.shape[1]):
        RV[:, t] = np.mean(log_returns[:, t-window:t]**2, axis=1) / delta
    
    return RV

def plot_exercise_boundary(params, t_plot=66):
    # Simulation parameters
    S0, x0 = 1.0, np.log(0.1)
    K = 0.0001
    T = 126/252
    Nsims = 10000
    
    # Simulate paths
    S, x = simulate_sv_paths(S0, x0, params['x_bar'], params['theta'], 
                           params['gamma_x'], params['rho'], params['rF'], 
                           params['muS'], T, params['delta'], Nsims)
    RV = compute_realized_variance(S)
    
    # Plot exercise boundary at t=66 (60 days before maturity)
    plt.figure(figsize=(12, 5))
    
    # Plot 1: Without paths
    plt.subplot(1, 2, 1)
    plt.scatter(x[:, t_plot], RV[:, t_plot], alpha=0.5, s=1)
    plt.xlabel('Log Volatility (x)')
    plt.ylabel('Realized Variance (RV)')
    plt.title(f'Exercise Boundary (Without Paths) at t={t_plot}')
    
    # Plot 2: With paths
    plt.subplot(1, 2, 2)
    X = np.column_stack([np.ones_like(x[:, t_plot]), 
                        x[:, t_plot], 
                        RV[:, t_plot], 
                        x[:, t_plot]**2, 
                        x[:, t_plot]*RV[:, t_plot], 
                        RV[:, t_plot]**2])
    
    # Compute continuation value
    Y = np.maximum(RV[:, t_plot+1] - K, 0) * np.exp(-params['rF'] * params['delta'])
    reg = LinearRegression().fit(X, Y)
    continuation_value = reg.predict(X)
    
    # Plot exercise regions
    exercise_region = np.maximum(RV[:, t_plot] - K, 0) > continuation_value
    plt.scatter(x[~exercise_region, t_plot], RV[~exercise_region, t_plot], 
               c='blue', alpha=0.5, s=1, label='Continue')
    plt.scatter(x[exercise_region, t_plot], RV[exercise_region, t_plot], 
               c='red', alpha=0.5, s=1, label='Exercise')
    plt.xlabel('Log Volatility (x)')
    plt.ylabel('Realized Variance (RV)')
    plt.title(f'Exercise Boundary (With Paths) at t={t_plot}')
    plt.legend()
    
    plt.tight_layout()
    plt.show()
    
    return S, RV

# Parameters
params = {
    'x_bar': np.log(0.1),
    'theta': 4,
    'gamma_x': 2,
    'rho': -0.9,
    'rF': 0.05,
    'muS': 0.10,
    'delta': 1/252,
    't': 66
}

# Plot exercise boundary
S, RV = plot_exercise_boundary(params)

# Compute option value and confidence interval
payoffs = np.maximum(RV[:, -1] - 0.0001, 0)
option_value = np.mean(payoffs) * np.exp(-params['rF'] * 126/252)
std_error = np.std(payoffs) / np.sqrt(10000)
ci_lower = option_value - 1.96 * std_error
ci_upper = option_value + 1.96 * std_error

print(f"Option Value: {option_value:.6f}")
print(f"95% CI: ({ci_lower:.6f}, {ci_upper:.6f})")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Parameters
T = 24  # Total periods
S_bar = 1.0  # Mean price of the stock
rho = 0.99  # Autoregressive coefficient
sigma = 0.01  # Standard deviation of price innovations
lambda_ = 0.01  # Price impact coefficient
X_total = 1.0  # Total shares to sell

# Grids
num_price_points = 100  # Number of discretization points for price
num_share_points = 50  # Number of discretization points for shares (control)

# Create price grid centered around S_bar with 3 standard deviation coverage
price_sd = sigma * np.sqrt((1 - rho**2) / (1 - rho))
price_min = S_bar - 5 * price_sd
price_max = S_bar + 5 * price_sd
price_grid = np.linspace(price_min, price_max, num_price_points)

# Shares grid from 0 to X_total
shares_grid = np.linspace(0, X_total, num_share_points + 1)

# Initialize value function (V) and policy (pi)
V = np.zeros((T + 1, num_price_points, num_share_points + 1))
pi = np.zeros((T, num_price_points, num_share_points + 1))

# Transition probabilities for price changes
transition_probs = np.zeros((num_price_points, num_price_points))
for i in range(num_price_points):
    current_price = price_grid[i]
    for j in range(num_price_points):
        next_price = price_grid[j]
        mean = rho * current_price + (1 - rho) * S_bar
        transition_probs[i, j] = np.exp(-(next_price - mean) ** 2 / (2 * sigma ** 2))
    transition_probs[i, :] /= np.sum(transition_probs[i, :])

plt.imshow(transition_probs, cmap='hot', interpolation='nearest', extent=[price_min, price_max, price_min, price_max], aspect='auto')
plt.colorbar()
plt.title('Transition Matrix Heatmap')
plt.xlabel('Next Price')
plt.ylabel('Current Price')
plt.show()

# Backward induction to solve for V and pi
for t in range(T - 1, -1, -1):
    for i in range(num_price_points):
        for x in range(num_share_points + 1):
            shares_remaining = shares_grid[x]
            for sell in range(x + 1):
                shares_to_sell = shares_grid[sell]
                immediate_profit = shares_to_sell * (price_grid[i] - lambda_ * shares_to_sell)
                future_profit = 0
                for j in range(num_price_points):
                    future_profit += transition_probs[i, j] * V[t + 1, j, x - sell]
                total_profit = immediate_profit + future_profit
                if total_profit > V[t, i, x]:
                    V[t, i, x] = total_profit
                    pi[t, i, x] = shares_to_sell

# Plot results
plt.figure(figsize=(10, 6))
for t in [0, 6, 12, 18]:
    plt.plot(price_grid, pi[t, :, num_share_points], label=f'Policy at t={t}')
plt.title('Optimal Selling Strategy Over Time')
plt.xlabel('Price')
plt.ylabel('Shares to Sell')
plt.legend()
plt.grid(True)
plt.show()

import numpy as np
import matplotlib.pyplot as plt

# Assuming the setup and V, pi from the previous explanation are already defined and computed
T = 24
mid_t = T // 2

# For (b) Optimal Policy as a function of St at t = T/2 and Wt = 0.5
shares_index_at_50_percent = np.searchsorted(shares_grid, 0.5)
optimal_policy_at_mid_t = pi[mid_t, :, shares_index_at_50_percent]

plt.figure(figsize=(10, 5))
plt.plot(price_grid, optimal_policy_at_mid_t)
plt.title(f"Optimal Policy at t={mid_t} with 50% Shares Remaining")
plt.xlabel("Stock Price $S_t$")
plt.ylabel("Shares to Sell")
plt.grid(True)
plt.show()

# For (c) Assuming St = 1
price_index_at_1 = np.argmin(np.abs(price_grid - 1))
optimal_policy_at_price_1 = pi[mid_t, price_index_at_1, :]

plt.figure(figsize=(10, 5))
plt.plot(shares_grid, optimal_policy_at_price_1)
plt.title(f"Optimal Policy at t={mid_t} and $S_t=1$")
plt.xlabel("Remaining Shares")
plt.ylabel("Shares to Sell")
plt.grid(True)
plt.show()

import numpy as np
import matplotlib.pyplot as plt

# Function to simulate stock prices
def simulate_stock_prices(S0, rho, sigma, T, num_paths):
    prices = np.zeros((num_paths, T+1))
    prices[:, 0] = S0
    for t in range(1, T+1):
        epsilon = np.random.normal(0, sigma, num_paths)
        prices[:, t] = rho * prices[:, t-1] + (1 - rho) * S_bar + epsilon
    return prices

# Parameters
num_paths = 100000
S0 = 1

# Simulate stock prices
simulated_prices = simulate_stock_prices(S0, rho, sigma, T, num_paths)

# Compute profits for both strategies
uniform_shares_to_sell = X_total / T
optimal_profits = np.zeros(num_paths)
uniform_profits = np.zeros(num_paths)

for i in range(num_paths):
    remaining_shares = X_total
    current_shares = X_total
    for t in range(T):
        # Find nearest price grid index for optimal policy
        price_index = np.argmin(np.abs(price_grid - simulated_prices[i, t]))
        if remaining_shares > 0:
            # Optimal policy shares to sell
            shares_index = np.searchsorted(shares_grid, remaining_shares, side='right') - 1
            shares_to_sell = pi[t, price_index, shares_index]
            optimal_profits[i] += shares_to_sell * (simulated_prices[i, t] - lambda_ * shares_to_sell)
            remaining_shares -= shares_to_sell
        
        # Uniform strategy
        uniform_profits[i] += uniform_shares_to_sell * (simulated_prices[i, t] - lambda_ * uniform_shares_to_sell)
        current_shares -= uniform_shares_to_sell

# Analysis
print("Average Profit - Optimal Policy:", np.mean(optimal_profits))
print("Average Profit - Uniform Selling:", np.mean(uniform_profits))
print("Performance difference depends on rho:", rho)

# Plotting results
plt.hist(optimal_profits, bins=50, alpha=0.5, label='Optimal Strategy')
plt.hist(uniform_profits, bins=50, alpha=0.5, label='Uniform Strategy')
plt.xlabel('Profit')
plt.ylabel('Frequency')
plt.legend()
plt.title('Profit Distribution of Trading Strategies')
plt.show()

def get_analytical_xt2(S_t3, W_t2, rho, lambda_, S_bar):
   """Analytical solution for optimal trading at T-2"""
   return (1/9) * W_t2 + (4*rho*(1 + rho - 0.5*rho**2)*(S_t3 - S_bar))/(9*lambda_)

# Get numerical solution at T-2
numerical_xt2 = pi[T-2]

# Calculate analytical solution for each price-shares combination
analytical_xt2 = np.zeros_like(numerical_xt2)
for i, s in enumerate(price_grid):
   for j, w in enumerate(shares_grid):
       analytical_xt2[i,j] = get_analytical_xt2(s, w, rho, lambda_, S_bar)

# Plot comparison for a few remaining share levels
share_levels = [0.25, 0.5, 0.75, 1.0]
plt.figure(figsize=(12, 8))
for w in share_levels:
   w_idx = np.searchsorted(shares_grid, w)
   plt.plot(price_grid, numerical_xt2[:,w_idx], '--', label=f'Numerical W={w:.2f}')

plt.title('T-2 Policy Comparison: Numerical Solution')
plt.xlabel('Stock Price')
plt.ylabel('Optimal Trade Size')
plt.legend()
plt.grid(True)
plt.show()